# CS225 Discussion  8
### Exercise 2
(i) Use a directed graph to represent the search for the n-queens problem discussed in the lectures. Implement the backtracking-based algorithm to solve this problem.  
(ii) Implement the branch-and-bound strategy for the travelling salesman problem.  
(iii) Implement a backtracking algorithm to ﬁnd a path out of a maze.


In [11]:
import numpy as np

In [3]:
# Modified DiGraph class from Graph Class
class DiGraph:
    def __init__(self,edges=[]):
        self.vertexList = VertexList(edges)
        for e in edges:
            self.addEdge(e)
    def addEdge(self,edge):
        # locate the related vertex according to the edge given
        vertex = self.vertexList.locate(edge[0]) # edge[0] is the incoming vertex
        edgelist = vertex.edges # get the edgelist object for this incoming vertex
        if edgelist != None:
            # add outgoing vertex to the edgelist
            edgelist.add(edge[1])
        else:
            # construct a new Edgelist object if this vertex has no edgelist yet
            edgelist = EdgeList(edge[1])
        vertex.setEdges(edgelist)
    def __iter__(self):
        vertices = self.vertexList
        for v in vertices:
            x = vertices.locate(v)
            y = x.edges
            if y != None:
                for z in y:
                    yield (v,z)
    def insertVertex(self,item):
        if not (item in self.vertexList):
            self.vertexList.append(item)
    def deleteVertex(self,item):
        return self.vertexList.remove(item)
    def insertEdge(self,edge):
        self.vertexList.addVertex(edge)
        self.addEdge(edge)
    def deleteEdge(self,edge):
        self.__deleteEdge(edge)
    def __deleteEdge(self,edge):
        if not (edge[0] in self.vertexList):
            print("There is no edge", edge)
            return False
        vertexlocation = self.vertexList.locate(edge[0])
        edgelist = vertexlocation.getEdges()
        if edgelist == None:
            print("There is no edge", edge)
            return False
        res = edgelist.remove(edge[1])
        if res == False:
            print("There is no edge", edge)
        return res
    def outgoingEdges(self,item):
        vertex = self.vertexList.locate(item)
        if vertex == None:
            print("There is no vertex", item)
            return []
        edgelist = vertex.getEdges()
        if edgelist == None:
            return []
        res = []
        for v in edgelist:
            res.append((item,v))
        return res
            # yield (item,v) # If we replace the above two lines with this line, then this methods works as an iterator.

# Definition of VertexList Class, a linked list
class VertexList:
    class  __Vertex:
        def __init__(self,item,next=None,previous=None):
            self.item=item # the vertex content
            self.next=next
            self.previous=previous
            self.edges=None # vertices related to this vertex
        def getItem(self):
            return self.item
        def getNext(self):
            return self.next
        def getPrevious(self):
            return self.previous
        def getEdges(self):
            return self.edges
        def setItem(self,item):
            self.item = item
        def setNext(self,next):
            self.next = next
        def setPrevious(self,previous):
            self.previous = previous
        def setEdges(self,edge):
            self.edges = edge
    
    def __init__(self,edges=[]):
        self.dummy = VertexList.__Vertex(None,None,None)
        self.numVertices = 0
        self.dummy.setNext(self.dummy)
        self.dummy.setPrevious(self.dummy)
        for e in edges:
            self.addVertex(e)
    def __iter__(self):
        cursor = self.dummy
        for i in range(self.numVertices):
            cursor = cursor.getNext()
            yield cursor.getItem()
    def append(self,item):
        lastVertex = self.dummy.getPrevious()
        newVertex = VertexList.__Vertex(item,self.dummy,lastVertex)
        lastVertex.setNext(newVertex)
        self.dummy.setPrevious(newVertex)
        self.numVertices += 1
    def __contains__(self,item):
        cursor = self.dummy
        for i in range(self.numVertices):
            cursor = cursor.getNext()
            vertex = cursor.getItem()
            if vertex == item:
                return True
        return False
    # locate the vertex location using its vertex value
    def locate(self,vertex): 
        cursor = self.dummy
        for i in range(self.numVertices):
            cursor = cursor.getNext()
            item = cursor.getItem()
            if vertex == item:
                return cursor
    # add new vertex if possible for the new edge
    def addVertex(self,edge):
        node1 = edge[0]
        node2 = edge[1]
        if not (node1 in self):
            self.append(node1)
        if not (node2 in self):
            self.append(node2)
    # remove a vertex
    def remove(self,item):
        cursor = self.dummy
        location = None
        for i in range(self.numVertices):
            cursor = cursor.getNext()
            vertex = cursor.getItem()
            edgelist = cursor.edges
            if edgelist != None:
                
                if item in edgelist:
                    print(item, "cannot be deleted, as it appears in an edge.")
                    return False
            if vertex == item:
                location = cursor
        if location == None:
            print(item, "is not a vertex.")
            return False
        nextVertex = location.getNext()
        prevVertex = location.getPrevious()
        prevVertex.setNext(nextVertex)
        nextVertex.setPrevious(prevVertex)
        self.numVertices -= 1
        return True
    def index(self,item):
        cursor = self.dummy
        for i in range(self.numVertices):
            cursor = cursor.getNext()
            if cursor.getItem() == item:
                return i
        return -1
    def getlength(self):
        return self.numVertices
    
# Definition of EdgeList Class, also a linked list
class EdgeList:
    class __Edge:
        def __init__(self,item,next=None,previous=None):
            self.item=item
            self.next=next
            self.previous=previous
        def getItem(self):
            return self.item
        def getNext(self):
            return self.next
        def getPrevious(self):
            return self.previous
        def setItem(self,item):
            self.item = item
        def setNext(self,next):
            self.next = next
        def setPrevious(self,previous):
            self.previous = previous
    
    def __init__(self,edge):
        self.first = EdgeList.__Edge(edge,None,None)
        self.first.setNext(self.first)
        self.first.setPrevious(self.first)
        self.numEdges = 1
    def add(self,edge):
        lastEdge = self.first.getPrevious()
        newEdge = EdgeList.__Edge(edge,self.first,lastEdge)
        lastEdge.setNext(newEdge)
        self.first.setPrevious(newEdge)
        self.numEdges += 1
    def __iter__(self):
        cursor = self.first
        for i in range(self.numEdges):
            yield cursor.getItem()
            cursor = cursor.getNext()
    def __contains__(self,item):
        cursor = self.first
        for i in range(self.numEdges):
            vertex = cursor.getItem()
            if vertex == item:
                return True
            cursor = cursor.getNext()
        return False
    def remove(self,item):
        cursor = self.first
        for i in range(self.numEdges):
            vertex = cursor.getItem()
            if vertex == item:
                nextVertex = cursor.getNext()
                prevVertex = cursor.getPrevious()
                prevVertex.setNext(nextVertex)
                nextVertex.setPrevious(prevVertex)
                self.numEdges -= 1
                if (cursor == self.first):
                    self.first = nextVertex
                return True
            cursor = cursor.getNext()
        return False


In [6]:
# 2.(i) n-queens problem
def nQueens(n):
    # Special case
    if n == 1:
        return [(1,1)]
    # Build the directed graph
    # Each vertex is a tuple (i, sigma(i)), where i is the number of row,
    # and sigma(i) is the column where a queen is placed in this row.
    g = DiGraph()
    for i in range(1, n):
        for j in range(1, n+1):
            for k in range(1, n+1):
                if j == k:
                    continue
                g.insertEdge(((i, j), (i+1, k)))
    
    # Find the first possible solution for the problem
    for col in range(1, n+1):
        visited = [(1, col)] # List of visited vertices
        results = BackTrack((1, col), visited, g)
        if results != None:
            print(results)
            return

# Recursive backtracking function
def BackTrack(curr_v, visited, graph):
    out_list = graph.outgoingEdges(curr_v)
    # Terminating condition
    if out_list == []:
        return visited
    
    # Recursive process
    for edge in out_list:
        # Check validity of the choice
        if not isValid(edge[1], visited):
            continue
        visited.append(edge[1])
        # Try to do recursion
        result_tmp = BackTrack(edge[1], visited, graph)
        if result_tmp == None:
            del visited[-1]
            continue
        else:
            return result_tmp
    return None

# Validity Checking Function
def isValid(curr_v, visited):
    for other_v in visited:
        # Column threats
        if other_v[1] == curr_v[1]:
            return False
        # Diagonal threats
        if other_v[1] + other_v[0] == curr_v[1] + curr_v[0]:
            return False
        if other_v[1] - other_v[0] == curr_v[1] - curr_v[0]:
            return False
    return True

In [5]:
# 2.(i) Test
nQueens(8)

[(1, 1), (2, 5), (3, 8), (4, 6), (5, 3), (6, 7), (7, 2), (8, 4)]


In [120]:
# 2.(ii) Travelling Salesman Problem
def TSP(n):
    # Represent the weighted directed graph as a matrix
    # Assume the cost is between 1 and 20
    mat = np.random.randint(1, 21, (n, n))
    for i in range(n):
        mat[i, i] = 0
    print("Cost Matrix:\n", mat)
    
    # Calculate the cost for visiting for each vertex
    cost_visit = Cost4Visit(mat, n)
    # We use the index from 0 to n-1 to represent vertex 1 to n
    path, cost = BranchNBound([0], mat, cost_visit, n, np.inf) # Always start from vertex 1 (index 0)
    path = [v+1 for v in path]
    print("Path:\n", path) # Change the index to vertex ID
    print("Cost:\n", cost)

# Cost for visiting
def Cost4Visit(mat, n):
    cost_visit = [0] * n
    # Only for n x n matrix
    # Set the diagonal to be larger than other entries
    all_max = np.max(mat)
    for i in range(n):
        mat[i, i] = all_max + 1
        dep_min = 0.5 * min(mat[i, :])
        arr_min = 0.5 * min(mat[:, i])
        cost_visit[i] = dep_min + arr_min
    return cost_visit

# Branch and Bound Process
def BranchNBound(visited, mat, cost_visit, n, min_cost):
    # Terminating Condition
    if len(visited) == n - 1:
        # Find the last vertex
        tmp_mat = np.zeros(n)
        tmp_mat[visited] = 1
        last_idx = np.argwhere(tmp_mat == 0)[0,0]
        # Calculate the total real cost
        path = visited.copy()
        path.append(last_idx)
        path.append(path[0])
        cost_total = 0
        for i in range(n):
            cost_total += mat[path[i], path[i+1]]
        return path, cost_total
    
    # Intermediate Condition
    tmp_mat = np.zeros(n)
    tmp_mat[visited] = 1
    out_list = np.argwhere(tmp_mat == 0)[:,0]
    # Calculate the lower bound for each possible choice
    lower_bound = np.zeros(len(out_list))
    for i in range(len(out_list)):
        next_v = out_list[i]
        # Real cost of visited vertices
        for j in range(len(visited) - 1):
            lower_bound[i] += mat[visited[j], visited[j+1]]
        lower_bound[i] += mat[visited[-1], next_v]
        # Estimated cost of future vertices
        lower_bound[i] += 0.5 * min(mat[next_v, :])
        lower_bound[i] += 0.5 * min(mat[:, 0])
        for other_v in out_list:
            if other_v == next_v:
                continue
            lower_bound[i] += cost_visit[other_v]
    
    # Search for next layer according to the order of lower bounds
    search_order = np.argsort(lower_bound)
    path = [[]] * len(out_list)
    cost = [np.inf] * len(out_list)
    count = 0
    for next_idx in search_order:
        # Rule out large lower bounds
        if np.ceil(lower_bound[next_idx]) >= min_cost:
            count += 1
            continue
        # Recurvisely compute the cost
        next_v = out_list[next_idx]
        visited.append(next_v)
        path[count], cost[count] = BranchNBound(visited, mat, cost_visit, n, min_cost)
        # Update the minimum cost
        if cost[count] < min_cost:
            min_cost = cost[count]
        del visited[-1]
        count += 1
    
    # Pick the path with smallest cost
    min_cost_idx = cost.index(min(cost))
    return path[min_cost_idx], cost[min_cost_idx]

In [123]:
# 2.(ii) Test
TSP(5)

Cost Matrix:
 [[ 0  5  9 12 16]
 [18  0  4 19 17]
 [13 10  0 13 19]
 [ 7 10 10  0  7]
 [11 14  2 12  0]]
Path:
 [1, 2, 3, 4, 5, 1]
Cost:
 40


(iii)  
We can treat each position of the maze as a vertex, which has *at most* $4$ edges connecting it with its up/down/left/right neighbor.  
Then the backtracking problem can be turned to DFS.

### Exercise 3
(i) Implement Kruskal's algorithm for the construction of a minimum spanning tree.  
(ii) Implement a partition data structure to support your implementation in (i). For this use a list $L$, in which entries correspond to vertices. If $L[i] = j$, then vertex $i$ is in the connected component of $j$. In this way each connected component is represented as a tree, so can easily check if the endpoints of the current edge under consideration are already in the same connected component. Show how to realise the merging of connected components.


In [12]:
# 3.(i) Original Kruskal's Algorithm
# This function will generate a random-weighted complete graph with n vertices and output the MST
def Kruskal(n):
    # Generate a random undirected graph
    # The range of weight is [1, 20]
    mat = np.random.randint(1, 21, (n, n))
    edge_lst = [] # Each edge is stored in the form of (cost, vertex1, vertex2), where vertex1 < vertex2
    # Set the cost matrix symmetric about the diagonal
    for i in range(n):
        for j in range(n):
            if i == j:
                mat[i, j] = 0
            elif i > j:
                mat[i, j] = mat[j, i]
            else:
                edge_lst.append((mat[i, j], i, j))
    print("Cost Matrix:\n", mat)
    
    # Sort the edge list
    edge_lst=sorted(edge_lst,key=lambda x:x[0]) 
    print("Edge List:\n", edge_lst)
    
    # We use index from 0 to n - 1
    CC_lst = [[i] for i in range(n)]
    MST_edges = []
    while len(CC_lst) > 1:
        curr_edge = edge_lst.pop(0)
        for CC in CC_lst:
            if curr_edge[1] in CC:
                CC_1 = CC.copy()
            if curr_edge[2] in CC:
                CC_2= CC
        # Merge two connected component
        if CC_1 == CC_2:
            continue
        CC_lst.remove(CC_1)
        CC_lst.remove(CC_2)
        CC_1.extend(CC_2)
        CC_lst.append(CC_1)
        MST_edges.append((curr_edge[1], curr_edge[2]))
    
    # Output the result
    print("MST:\n", MST_edges)

In [13]:
# 3.(i) Test
Kruskal(7)

Cost Matrix:
 [[ 0  5  5 14  6 18  5]
 [ 5  0  2 18  6 13 14]
 [ 5  2  0  6  7 16  9]
 [14 18  6  0 19 14 14]
 [ 6  6  7 19  0  4 12]
 [18 13 16 14  4  0 19]
 [ 5 14  9 14 12 19  0]]
Edge List:
 [(2, 1, 2), (4, 4, 5), (5, 0, 1), (5, 0, 2), (5, 0, 6), (6, 0, 4), (6, 1, 4), (6, 2, 3), (7, 2, 4), (9, 2, 6), (12, 4, 6), (13, 1, 5), (14, 0, 3), (14, 1, 6), (14, 3, 5), (14, 3, 6), (16, 2, 5), (18, 0, 5), (18, 1, 3), (19, 3, 4), (19, 5, 6)]
MST:
 [(1, 2), (4, 5), (0, 1), (0, 6), (0, 4), (2, 3)]


In [16]:
# 3.(ii) Modified Kruskal's Algorithm
# This function will generate a random-weighted complete graph with n vertices and output the MST
def Kruskal_M(n):
    # Generate a random undirected graph
    # The range of weight is [1, 20]
    mat = np.random.randint(1, 21, (n, n))
    edge_lst = [] # Each edge is stored in the form of (cost, vertex1, vertex2), where vertex1 < vertex2
    # Set the cost matrix symmetric about the diagonal
    for i in range(n):
        for j in range(n):
            if i == j:
                mat[i, j] = 0
            elif i > j:
                mat[i, j] = mat[j, i]
            else:
                edge_lst.append((mat[i, j], i, j))
    print("Cost Matrix:\n", mat)
    
    # Sort the edge list
    edge_lst=sorted(edge_lst,key=lambda x:x[0]) 
    print("Edge List:\n", edge_lst)
    
    # We use index from 0 to n - 1
    L = [-(i+1) for i in range(n)] # Modification
    print(L)
    MST_edges = []
    while len(set(L)) > 1:
        curr_edge = edge_lst.pop(0)
        # Modification
        if L[curr_edge[1]] < 0:
            if L[curr_edge[2]] < 0:
                L[curr_edge[1]] = curr_edge[1]
                L[curr_edge[2]] = curr_edge[1]
            else:
                L[curr_edge[1]] = L[curr_edge[2]]
        else:
            if L[curr_edge[2]] < 0:
                L[curr_edge[2]] = L[curr_edge[1]]
            elif L[curr_edge[2]] != L[curr_edge[1]]:
                tmp = L[curr_edge[2]]
                # update for the second connected component
                for i in range(n):
                    if L[i] == tmp:
                        L[i] = L[curr_edge[1]]
            else:
                continue
        print(L)
        MST_edges.append((curr_edge[1], curr_edge[2]))
    
    # Output the result
    print("MST:\n", MST_edges)

In [17]:
# 3.(ii) Test
Kruskal_M(7)

Cost Matrix:
 [[ 0 20  9  5  3  4 12]
 [20  0 15 20  9  8 17]
 [ 9 15  0 16 10 10 13]
 [ 5 20 16  0  1 20  1]
 [ 3  9 10  1  0 15 18]
 [ 4  8 10 20 15  0 18]
 [12 17 13  1 18 18  0]]
Edge List:
 [(1, 3, 4), (1, 3, 6), (3, 0, 4), (4, 0, 5), (5, 0, 3), (8, 1, 5), (9, 0, 2), (9, 1, 4), (10, 2, 4), (10, 2, 5), (12, 0, 6), (13, 2, 6), (15, 1, 2), (15, 4, 5), (16, 2, 3), (17, 1, 6), (18, 4, 6), (18, 5, 6), (20, 0, 1), (20, 1, 3), (20, 3, 5)]
[-1, -2, -3, -4, -5, -6, -7]
[-1, -2, -3, 3, 3, -6, -7]
[-1, -2, -3, 3, 3, -6, 3]
[3, -2, -3, 3, 3, -6, 3]
[3, -2, -3, 3, 3, 3, 3]
[3, 3, -3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3]
MST:
 [(3, 4), (3, 6), (0, 4), (0, 5), (1, 5), (0, 2)]
